# Fraud Detection using LightGBM and SMOTE

### Introduction
We use the dataset of [ULB Credit Card Dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud) to train our frauld detection model. In this notebook, we will train our model using [LighGBM](https://microsoft.github.io/SynapseML/docs/next/features/lightgbm/LightGBM%20-%20Overview/) and [SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html) algorithms, which refers to some execellent work listed as below:

* **Fraud detection handbook**: https://fraud-detection-handbook.github.io/fraud-detection-handbook/Foreword.html
* **AWS creditcard fraud detector**: https://github.com/awslabs/fraud-detection-using-machine-learning/blob/master/source/notebooks/sagemaker_fraud_detection.ipynb
* **Creditcard fraud detection predictive models**: https://www.kaggle.com/code/gpreda/credit-card-fraud-detection-predictive-models

In [ ]:
import pyspark
import yaml
import numpy as np
import pandas as pd
import warnings

from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
def init_spark():
    spark = pyspark.sql.SparkSession.builder\
            .appName("Fraud Detection-LightGBM") \
            .config("spark.executor.memory","8G") \
            .config("spark.executor.instances","4") \
            .config("spark.executor.cores", "4") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.4") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()
    sc = spark.sparkContext
    print(sc.version)
    print(sc.applicationId)
    print(sc.uiWebUrl)
    return spark

def load_config(path):
    params = dict()
    with open(path, 'r') as stream:
        params = yaml.load(stream, Loader=yaml.FullLoader)
    return params

def read_dataset(spark, data_path):
    dataset = spark.read.format("csv")\
      .option("header",  True)\
      .option("inferSchema",  True)\
      .load(data_path)  
    return dataset

def get_vectorassembler(dataset, features='features', label='label'):
    featurizer = VectorAssembler(
        inputCols = feature_cols,
        outputCol = 'features',
        handleInvalid = 'skip'
    )
    dataset = featurizer.transform(dataset)[label, features]
    return dataset

In [ ]:
spark = init_spark()

### Train detection model using LightGBM

You should replace ``{MY_S3_BUCKET}``, ``{TRAIN_S3_PATH}`` and ``{TEST_S3_PATH}`` with actual values before executing code cells containing these placeholders.

In [ ]:
train_file_path = 's3://{MY_S3_BUCKET}/{TRAIN_S3_PATH}'
test_file_path = 's3://{MY_S3_BUCKET}/{TEST_S3_PATH}'
fg_train_dataset = read_dataset(spark, train_file_path)
fg_test_dataset = read_dataset(spark, test_file_path)

In [ ]:
fg_train_dataset.printSchema()

The dataset contains only numerical features, because the original features have been transformed using PCA. As a result, the dataset contains 28 PCA components, V1-V28, and two features that haven't been transformed, `Amount` and `Time`. Amount refers to the transaction amount, and Time is the seconds elapsed between any transaction in the data and the first transaction.Moreover, The `Class` column corresponds to whether or not a transaction is fraudulent. 


> https://github.com/awslabs/fraud-detection-using-machine-learning

In [ ]:
fg_train_dataset.groupby('Class').count().toPandas()

In [ ]:
fg_test_dataset.groupby('Class').count().toPandas()

In [ ]:
feature_cols = fg_train_dataset.columns[:-1]
feature_cols

In [ ]:
label_col = fg_train_dataset.columns[-1]
label_col

In [ ]:
train_data = get_vectorassembler(fg_train_dataset, label=label_col, features='features')
test_data = get_vectorassembler(fg_test_dataset, label=label_col, features='features')

In [ ]:
train_data.limit(10).toPandas()

We split train dataset for model training and validation

In [ ]:
train, valid = train_data.randomSplit([0.90, 0.10], seed=2022)

In [ ]:
train.count()

In [ ]:
valid.count()

### Train model
Here we are using `isUnbalance=True`, please refer to [LightGBM docs](https://mmlspark.blob.core.windows.net/docs/0.18.1/pyspark/mmlspark.lightgbm.html) for the description of this parameter. Moreover, we will test the model performance by using multiple metrics, such as `AUC`, `KS`, `Balanced accuracy`, `Cohen's kappa` and `Confusion Matrix`.

In [ ]:
model_params =  {
    'boostingType':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'numLeaves': 7,
    'maxDepth': 4,
    'minDataInLeaf': 100,
    'maxBin': 100,
    'minGainToSplit': 0.0,
    'featureFraction': 0.7,
    'baggingFraction': 0.9,
    'baggingFreq': 1,
    'learningRate': 0.01,
    'numIterations': 300,
    'earlyStoppingRound': 100,
    'verbosity':1,
    'numThreads':16,
}

def train_lightgbm(train_dataset, feature_col, label_col, model_params):
    from synapse.ml.lightgbm import LightGBMClassifier
    model = LightGBMClassifier(isProvideTrainingMetric=True, featuresCol=feature_col, labelCol=label_col, isUnbalance=True, **model_params)
    model = model.fit(train_dataset)
    return model

def evaluate(predictions, label_col, metricName="areaUnderROC"):
    evaluator = BinaryClassificationEvaluator(labelCol=label_col, metricName="areaUnderROC")
    return evaluator.evaluate(predictions)

model = train_lightgbm(train, 'features', 'Class', model_params)

In [ ]:
print("train dataset prediciton:")
predictions = model.transform(train)
print("train dataset auc:", evaluate(predictions, label_col))

In [ ]:
print("validation dataset prediciton:")
predictions = model.transform(valid)
print("validation dataset auc:", evaluate(predictions, label_col))

In [ ]:
importance_df = (
    pd.DataFrame({
        'feature_name': feature_cols,
        'importance_gain': model.getFeatureImportances('gain'),
        'importance_split': model.getFeatureImportances('split'),
    })
    .sort_values('importance_gain', ascending=False)
    .reset_index(drop=True)
)
print(importance_df)

In [ ]:
model = train_lightgbm(train_data, 'features', 'Class', model_params)

In [ ]:
print("test dataset prediciton:")
predictions = model.transform(test_data)
print("test dataset auc:", evaluate(predictions, label_col))

In [ ]:
predictionAndLabels = predictions.select('prediction', F.col(label_col).cast(DoubleType()))\
                                 .withColumnRenamed(label_col, 'label')
metrics = MulticlassMetrics(predictionAndLabels.rdd)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(metrics.confusionMatrix().toArray(), 
           xticklabels=['Not Fraud', 'Fraud'],
           yticklabels=['Not Fraud', 'Fraud'],
           linewidths=5, fmt='g', annot=True)

In [ ]:
import sys
import matplotlib.pyplot as plt

sys.path.append('../../') 
from common.ks_utils import ks_2samp, ks_curve

label = np.array(predictions.select(label_col).collect()).reshape(-1).astype(np.float32)
prediction = np.array(predictions.select('probability').collect())[:, :, 1].reshape(-1)
print('label: ', label[0:10])
print('prediction: ', prediction[0:10])

ks = ks_2samp(label, prediction)
print("KS statistic: ", ks.statistic)
ks_curve(label, prediction)
plt.show()

In [ ]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

# scikit-learn expects 0/1 predictions, so we threshold our raw predictions
y_preds = np.where(prediction > 0.5, 1, 0)
print("Balanced accuracy = {}".format(balanced_accuracy_score(label, y_preds)))
print("Cohen's Kappa = {}".format(cohen_kappa_score(label, y_preds)))

### SMOTE
Now we have a baseline model using LightGBM. Let us will verify the over sampling tricks on traning data that using SMOTE. Moreover, we will test the model performance by using multiple metrics, such as `AUC`, `KS`, `Balanced accuracy`, `Cohen's kappa` and `Confusion Matrix`. You should replace ``{MY_S3_BUCKET}``, ``{TRAIN_SMOTE_S3_PATH}`` with actual values before executing code cells containing these placeholders.

In [ ]:
train_file_path = 's3://{MY_S3_BUCKET}/{TRAIN_SMOTE_S3_PATH}'
fg_train_smote_dataset = read_dataset(spark, train_file_path)
train_smote_data = get_vectorassembler(fg_train_smote_dataset, label=label_col, features='features')
train_smote_data.groupby('Class').count().toPandas()

In [ ]:
model = train_lightgbm(train_smote_data, 'features', 'Class', model_params)

In [ ]:
print("test dataset prediciton:")
predictions = model.transform(test_data)
print("test dataset auc:", evaluate(predictions, label_col))

In [ ]:
predictionAndLabels = predictions.select('prediction', F.col(label_col).cast(DoubleType()))\
                                 .withColumnRenamed(label_col, 'label')
metrics = MulticlassMetrics(predictionAndLabels.rdd)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(metrics.confusionMatrix().toArray(), 
           xticklabels=['Not Fraud', 'Fraud'],
           yticklabels=['Not Fraud', 'Fraud'],
           linewidths=5, fmt='g', annot=True)

In [ ]:
import sys
import matplotlib.pyplot as plt

sys.path.append('../../') 
from common.ks_utils import ks_2samp, ks_curve

label = np.array(predictions.select(label_col).collect()).reshape(-1).astype(np.float32)
prediction = np.array(predictions.select('probability').collect())[:, :, 1].reshape(-1)
print('label: ', label[0:10])
print('prediction: ', prediction[0:10])

ks = ks_2samp(label, prediction)
print("KS statistic: ", ks.statistic)
ks_curve(label, prediction)
plt.show()

In [ ]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

# scikit-learn expects 0/1 predictions, so we threshold our raw predictions
y_preds = np.where(prediction > 0.5, 1, 0)
print("Balanced accuracy = {}".format(balanced_accuracy_score(label, y_preds)))
print("Cohen's Kappa = {}".format(cohen_kappa_score(label, y_preds)))

In [ ]:
spark.stop()